In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
df = pd.read_csv('MoviesOnStreamingPlatforms_updated.csv')

In [3]:
#A dataframe with all the types of movie genres
genres = df['Genres'].unique().tolist()
genres = DataFrame (genres, columns=['Genres'])

In [4]:
#There are a lot of different genres. So in order for the user to know the genres to look for, we are going to sample 10 of them as reference.

In [5]:
genres.sample(10)

,Genres
177,"Animation,Adventure,Comedy,Family,Fantasy,Mystery"
1326,"Animation,Comedy,Drama,Family,Music,Musical,Ro..."
156,"Action,Crime,Horror,Sci-Fi,Thriller"
1007,"Biography,Musical"
1321,"Comedy,Family,Fantasy,Horror,Mystery"
820,"Action,Crime,Drama,History,Thriller"
632,"Animation,Short,Adventure,Comedy,Family,Fantas..."
267,Romance
245,"Action,Crime,Sci-Fi,Thriller"
506,"Fantasy,Horror,Mystery,Thriller"


In [6]:
genre = input('What kind of genre are you interested in? [Warning: case sensitive] ')

What kind of genre are you interested in? [Warning: case sensitive]  Thriller


In [7]:
filter = df['Genres'].str.contains(genre, na=False)

In [8]:
movies = df[filter]

In [9]:
#No Musical!

In [10]:
avoid_musical = ~movies['Genres'].str.contains('Musical', na=False)

In [11]:
movies = movies[avoid_musical]

In [12]:
#Picking the columns we need
col = ['Title', 'Year', 'IMDb', 'Rotten Tomatoes', 'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Runtime']

In [13]:
movies = movies[col]
movies.dropna(inplace=True) #Drop any NaN

In [14]:
#The votes are not numerical numbers, but strings like 'x/100' or 'x/10'. 
#Picking only the first two digits from the strings, then turning them into tenths.

In [15]:
movies['IMDb'] = movies['IMDb'].str[0:3]
movies['Rotten Tomatoes'] = movies['Rotten Tomatoes'].str[0:2]
movies['Rotten Tomatoes'] = movies['Rotten Tomatoes'].astype(int)
movies['Rotten Tomatoes'] /= 10
movies['IMDb'] = movies['IMDb'].astype(float)

In [16]:
#The Rating is an average of IMDB and Rotten Tomatoes' votes

In [17]:
movies['Rating'] = (movies['IMDb']+movies['Rotten Tomatoes'])/2
movies['Rating'] = movies['Rating'].round(2)

In [18]:
movies['Runtime'] = movies['Runtime'].astype(int)

In [19]:
#Selecting the column that are important for the analysis
col = ['Title', 'Year', 'Runtime', 'Rating', 'Netflix', 'Prime Video', 'Disney+']

In [20]:
#Netflix
Netflix_df = movies.loc[movies['Netflix']==1]
col=['Title', 'Year', 'Runtime', 'Rating']
Netflix_df = Netflix_df[col]
Netflix_df.sort_values(by="Rating", ascending=False, inplace=True)

In [21]:
#AmazonPrimeVideo
Prime_df = movies.loc[movies['Prime Video']==1]
col=['Title', 'Year', 'Runtime', 'Rating']
Prime_df = Prime_df[col]
Prime_df.sort_values(by="Rating", ascending=False, inplace=True)

In [22]:
#Disney+
Disney_df = movies.loc[movies['Disney+']==1]
col=['Title', 'Year', 'Runtime', 'Rating']
Disney_df = Disney_df[col]
Disney_df.sort_values(by="Rating", ascending=False, inplace=True)

In [23]:
#The result of the analysis is going to be saved as an excel file. The file name is the same as the genre been chosen.

In [25]:
with pd.ExcelWriter(f'{genre}.xlsx') as file:
    Netflix_df.to_excel(file, sheet_name='Netflix', index=False) #Sheet for Netflix Movies
    Prime_df.to_excel(file, sheet_name='Prime Video', index=False) #Sheet for Prime Video Movies
    Disney_df.to_excel(file, sheet_name='Disney +', index=False) #Sheet for Disney+ Movies